In [80]:
import json
import re
import nltk

In [81]:
# This file contains 8600ish users Data
# In the form: {'MBTI Type', 'Social Media Posts'}
dataFile = open('formatted_data.json', 'r')

In [82]:
data = []

In [83]:
# loading the json data into a list
for line in dataFile:
    data.append(json.loads(line))

In [84]:
types = []
posts = []

In [85]:
# Separating data into 2 separate lists for preprocessing
# For the most part we process the 'post' data
# we skip the first element as that is only the label
for i in range(1, len(data)):
    types.append(data[i]['Type'])
    posts.append(data[i]['Post'])

In [86]:
# List shuffling to make sure our models are valid
# Commented out for debugging purposes
# import random
# SEED = 673

# random.seed(SEED)
# random.shuffle(types)
# random.shuffle(posts)

In [87]:
# Turning the posts from: 'post1|||post2|||post3'
#                     to: ['post1', 'post2', 'post3']
# expects a list of posts as strings
# returns a list of lists of posts
def vectorize_post_data(posts):
    for index in range(0, len(posts)):
        posts[index] = posts[index].split("|||")
        
    return posts
        
posts = vectorize_post_data(posts)

In [88]:
def remove_hyperTextLinksFromPosts(posts):
    for index in range(0, len(posts)):
        usable_post = [post for post in posts[index] if not re.search(r'^(.)*http(.)*$', post)]
        posts[index] = usable_post
    
    return posts
        
posts = remove_hyperTextLinksFromPosts(posts)

In [89]:
def remove_MBTIClassifiersFromPosts(posts):
    MBTI_regex = r'[\w]*(i|e)(s|n)(f|t)(p|j)[\w]*'
    for i in range(0, len(posts)):
        for j in range(0, len(posts[i])):
            posts[i][j] = re.sub(MBTI_regex, ' ', posts[i][j], flags=re.IGNORECASE)
        
    return posts
    
posts = remove_MBTIClassifiersFromPosts(posts)

In [90]:
from nltk.tokenize import RegexpTokenizer

def isInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

def tokenize_posts(posts):
    tokenizer = RegexpTokenizer(r'\w+')
    
    for i in range(0, len(posts)):
        user_words = []
        for j in range(0, len(posts[i])):
            post = tokenizer.tokenize(posts[i][j])
            for word in post:
                if not isInt(word):
                    user_words.append(word.lower())
        posts[i] = user_words
    
    return posts

posts = tokenize_posts(posts)

In [91]:
all_words_list = []

for user in posts:
    for word in user:
        all_words_list.append(word)


In [92]:
from collections import Counter

In [93]:
freq_list = Counter(all_words_list)
dictionary = freq_list.most_common(10000)

In [94]:
dictionary = list(zip(*dictionary))[0]

In [95]:
nums = range(0, 10000)
word_int = dict(zip(dictionary, nums))

In [96]:
x_vals = []

for user in posts:
    x_vals.append([word_int[x] for x in user if x in word_int.keys()])

In [112]:
intro_extro = [word[0:1] for word in types[0:len(types)]]
bin_intro_extro = []
for letter in intro_extro:
    if (letter == 'I'):
        bin_intro_extro.append(0)
    else:
        bin_intro_extro.append(1)

In [114]:
import numpy as np
x = np.array(x_vals)
test_data = x[:1500]
train_data = x[1500:]

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [115]:
from keras.utils.np_utils import to_categorical

y = np.asarray(bin_intro_extro).astype('float32')
y_test = y[:1500]
y_train = y[1500:]

In [116]:
x_val = x_train[:1500]
x_partial_train = x_train[1500:]

y_val = y_train[:1500]
y_partial_train = y_train[1500:]

In [117]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape = (10000,)))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [118]:
history = model.fit(x_partial_train, y_partial_train, epochs = 20, batch_size = 512, validation_data=(x_val, y_val))

Train on 5675 samples, validate on 1500 samples
Epoch 1/20
5675/5675 [==============================] - 1s 163us/step - loss: 0.5585 - acc: 0.7586 - val_loss: 0.5525 - val_acc: 0.7580
Epoch 2/20
5675/5675 [==============================] - 1s 96us/step - loss: 0.5004 - acc: 0.7718 - val_loss: 0.5346 - val_acc: 0.7580
Epoch 3/20
5675/5675 [==============================] - 1s 98us/step - loss: 0.4357 - acc: 0.7741 - val_loss: 0.5730 - val_acc: 0.7580
Epoch 4/20
5675/5675 [==============================] - 1s 97us/step - loss: 0.4092 - acc: 0.8137 - val_loss: 0.4911 - val_acc: 0.7700
Epoch 5/20
5675/5675 [==============================] - 1s 97us/step - loss: 0.3648 - acc: 0.8448 - val_loss: 0.5091 - val_acc: 0.7573
Epoch 6/20
5675/5675 [==============================] - 1s 98us/step - loss: 0.3215 - acc: 0.8742 - val_loss: 0.4990 - val_acc: 0.7747
Epoch 7/20
5675/5675 [==============================] - 1s 97us/step - loss: 0.2915 - acc: 0.8867 - val_loss: 0.4975 - val_acc: 0.7587
Epoch 

In [119]:
results = model.evaluate(x_test, y_test)
results

1500/1500 [==============================] - 0s 88us/step


[0.7747726391951243, 0.7759999998410543]

In [120]:
model = models.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape = (10000,)))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [121]:
model.fit(x_train, y_train, epochs = 13, batch_size = 512)

Epoch 1/13
7175/7175 [==============================] - 1s 133us/step - loss: 0.5752 - acc: 0.7620
Epoch 2/13
7175/7175 [==============================] - 1s 91us/step - loss: 0.4904 - acc: 0.7689
Epoch 3/13
7175/7175 [==============================] - 1s 114us/step - loss: 0.4340 - acc: 0.7689
Epoch 4/13
7175/7175 [==============================] - 1s 93us/step - loss: 0.4114 - acc: 0.7817
Epoch 5/13
7175/7175 [==============================] - 1s 86us/step - loss: 0.3695 - acc: 0.8256
Epoch 6/13
7175/7175 [==============================] - 1s 84us/step - loss: 0.3283 - acc: 0.8636
Epoch 7/13
7175/7175 [==============================] - 1s 83us/step - loss: 0.3042 - acc: 0.8780
Epoch 8/13
7175/7175 [==============================] - 1s 83us/step - loss: 0.2549 - acc: 0.9024
Epoch 9/13
7175/7175 [==============================] - 1s 87us/step - loss: 0.2447 - acc: 0.9017
Epoch 10/13
7175/7175 [==============================] - 1s 84us/step - loss: 0.1974 - acc: 0.9292
Epoch 11/13
7175/

In [122]:
results = model.evaluate(x_test, y_test)
results

1500/1500 [==============================] - 0s 125us/step


[0.6135375473499298, 0.7719999998410543]